# scRFE V6

In [84]:
# MENTION ONE VS ALL CLASSIFICATION in description

In [1]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
import random
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# read in anndata file 
adata = read_h5ad('/Users/madelinepark/Downloads/Kidney_facs.h5ad')

In [3]:
def filterNormalize (dataMatrix, classOfInterest):
    np.random.seed(644685)
    sc.logging.print_versions()
    sc.settings.verbosity = 3      
    sc.logging.print_versions()
    tiss = dataMatrix
    tiss.obs['n_counts'] = tiss.X.sum(axis=1).A1
    sc.pp.filter_cells(tiss, min_genes=250)
    sc.pp.filter_genes(tiss, min_cells=3)
    tiss = tiss[tiss.obs['n_counts'] > 1500, :]
    sc.pp.normalize_per_cell(tiss, counts_per_cell_after=1e5)
    sc.pp.log1p(tiss)
    tiss.raw = tiss
    tiss = tiss[tiss.obs[classOfInterest]!='nan']
    return tiss

In [4]:
filterNormalize(dataMatrix = adata, classOfInterest = 'age')

scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 
scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 
filtered out 7032 genes that are detected in less than 3 cells


Trying to set attribute `.obs` of view, making a copy.


View of AnnData object with n_obs × n_vars = 1215 × 15867 
    obs: 'FACS.selection', 'age', 'batch', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'cellid', 'free_annotation', 'method', 'mouse.id', 'plate', 'sex', 'subtissue', 'tissue', 'well', 'n_genes', 'n_counts', 'louvain', 'cluster_names'
    var: 'n_cells', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_ontology_class_colors', 'louvain', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'X_tsne'
    varm: 'PCs'

In [120]:
# goal: get labels on a per class basis that will go into randomForest function for y
def getLabels (dataMatrix, classOfInterest): 
    """
    Gets labels on a per class basis that will inputted to the randomForest function
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    
    Returns
    -------
    labelsDict : dict
        Dictionary with labels for each class 
    """
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)
    labelsDict = {}
    for label in np.unique(dataMatrix.obs[classOfInterest]):
        lists = []        
        for obs in dataMatrix.obs[classOfInterest]:
            if obs == label: 
                lists.append('A')
            else:
                lists.append('B')
        labelsDict[label] = lists #this is usually in line w if and else    
    return labelsDict

In [121]:
def makeOneForest (dataMatrix, classOfInterest, labelOfInterest, nEstimators = 5000, 
                   randomState = 0,  nJobs = -1, oobScore = True, Step = 0.2, Cv = 5): 
    """
    Builds and runs a random forest for one label in a class of interest
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    labelOfInterest : str
        The specific label within the class that the random forezt will run a 
        "one vs all" classification on
    nEstimators : int
        The number of trees in the forest
    randomState : int
        Controls random number being used
    nJobs : int
        The number of jobs to run in parallel
    oobScore : bool
        Whether to use out-of-bag samples to estimate the generalization accuracy
    Step : float
        Corresponds to percentage of features to remove at each iteration
    Cv : int
        Determines the cross-validation splitting strategy
        
    Returns
    -------
    feature_selected : list
        list of top features from random forest
    selector.estimator_.feature_importances_ : list
        list of top ginis corresponding to to features
    
    """
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)

    print('makeOneForest' + labelOfInterest)
    labelsDict = getLabels(dataMatrix, classOfInterest) 

    feat_labels = dataMatrix.var_names #this is equivalent of the genes
    X = dataMatrix.X
    y = labelsDict[labelOfInterest]
    print('Y')
    print(len(y))
    clf = RandomForestClassifier(n_estimators = nEstimators, random_state = randomState, 
                                 n_jobs = nJobs, oob_score = oobScore)
    selector = RFECV(clf, step = Step, cv = Cv)
    
    print('training...')
    clf.fit(X, y)
    selector.fit(X, y)
    feature_selected = feat_labels[selector.support_] 

    return feature_selected, selector.estimator_.feature_importances_ 

In [122]:
def resultWrite (classOfInterest, results_df, labelOfInterest,
                feature_selected, feature_importance):
    print ('result writing')
    print(results_df)
    
    column_headings = [] 
    column_headings.append(labelOfInterest)
    column_headings.append(labelOfInterest + '_gini')
    resaux = pd.DataFrame(columns = column_headings)
    resaux[labelOfInterest] = feature_selected
    resaux[labelOfInterest + '_gini'] = feature_importance
    resaux = resaux.sort_values(by = [labelOfInterest + '_gini'], ascending = False)
    resaux.reset_index(drop = True, inplace = True)

    results_df = pd.concat([results_df, resaux], axis=1)
    return results_df 

In [123]:
def scRFE(dataMatrix, classOfInterest, nEstimators = 5000, randomState = 0,  
                  nJobs = -1, oobScore = True, Step = 0.2, Cv = 5):
    """
    Builds and runs a random forest with one vs all classification for each label 
    for one class of interest
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    labelOfInterest : str
        The specific label within the class that the random forezt will run a 
        "one vs all" classification on
    nEstimators : int
        The number of trees in the forest
    randomState : int
        Controls random number being used
    nJobs : int
        The number of jobs to run in parallel
    oobScore : bool
        Whether to use out-of-bag samples to estimate the generalization accuracy
    Step : float
        Corresponds to percentage of features to remove at each iteration
    Cv : int
        Determines the cross-validation splitting strategy
        
    Returns
    -------
    results_df : pd.DataFrame
        Dataframe with results for each label in the class, formatted as 
        "label" for one column, then "label + gini" for the corresponding column
    
    """
    
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)
    results_df = pd.DataFrame()
    for labelOfInterest in np.unique(dataMatrix.obs[classOfInterest]): #for timeliness    
        print( 'scRFE' + labelOfInterest)
        
        feature_selected, feature_importance = makeOneForest(dataMatrix, 
                                                             classOfInterest, 
                          labelOfInterest = labelOfInterest)
    
        results_df = resultWrite (classOfInterest, results_df, 
                            labelOfInterest = labelOfInterest, 
                    feature_selected = feature_selected,  
                    feature_importance = feature_importance)
        print(results_df.shape)
    return results_df

In [124]:
pwd

'/Users/madelinepark/src3/scRFE/scripts'

# Run scRFE on kidney celltype, 1000 estimators, cv = 5

In [125]:
kidney1000scRFECelltypeReIndex = scRFE (dataMatrix = adata, 
                                        classOfInterest = 'cell_ontology_class')

scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 
scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 
filtered out 7032 genes that are detected in less than 3 cells


Trying to set attribute `.obs` of view, making a copy.
Trying to set attribute `.obs` of view, making a copy.


scRFEB cell
scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 
scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


Trying to set attribute `.obs` of view, making a copy.


filtered out 100 genes that are detected in less than 3 cells


...

Y
1215
training...
result writing
              B cell  B cell_gini Kidney-new-24m-cluster  \
0               Ly6d     0.028970                Sec14l3   
1              Ms4a1     0.025944                Gm16861   
2              Cd79a     0.025738                  Myom3   
3               Ccr7     0.024561                 Gltpd1   
4              Faim3     0.020408                 Gm5820   
5              Cd79b     0.020378                Slc46a3   
6      2010001M09Rik     0.014993                 Beta-s   
7               Spib     0.011915                   Thpo   
8               Cd19     0.011665          5730419I09Rik   
9                Blk     0.009107                 Slc2a5   
10     1810046K07Rik     0.008602                  Hook2   
11           Pou2af1     0.007951                  Sgip1   
12         Tnfrsf13c     0.007839                   Cdx4   
13              Cd37     0.007784          C730036E19Rik   
14              Scd1     0.007096                Fam149a   
15    

Trying to set attribute `.obs` of view, making a copy.
Trying to set attribute `.obs` of view, making a copy.


makeOneForestmesangial cell
scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 
scanpy==1.4.3 anndata==0.6.21 umap==0.3.9 numpy==1.16.4 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.21.2 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


Trying to set attribute `.obs` of view, making a copy.


Y
1215
training...
result writing
              B cell  B cell_gini Kidney-new-24m-cluster  \
0               Ly6d     0.028970                Sec14l3   
1              Ms4a1     0.025944                Gm16861   
2              Cd79a     0.025738                  Myom3   
3               Ccr7     0.024561                 Gltpd1   
4              Faim3     0.020408                 Gm5820   
5              Cd79b     0.020378                Slc46a3   
6      2010001M09Rik     0.014993                 Beta-s   
7               Spib     0.011915                   Thpo   
8               Cd19     0.011665          5730419I09Rik   
9                Blk     0.009107                 Slc2a5   
10     1810046K07Rik     0.008602                  Hook2   
11           Pou2af1     0.007951                  Sgip1   
12         Tnfrsf13c     0.007839                   Cdx4   
13              Cd37     0.007784          C730036E19Rik   
14              Scd1     0.007096                Fam149a   
15    

In [126]:
kidney1000scRFECelltypeReIndex.to_csv('kidney1000scRFECelltypeReIndex.csv')